In [48]:
import pandas as pd

In [ ]:
df = pd.read_csv('youtube_data.csv')

In [ ]:

df = pd.DataFrame(data_search[0]['items'])

for i in range(len(data_search)-1):
    df = pd.concat([df, pd.DataFrame(data_search[i+1]['items'])])
df = pd.DataFrame(df['snippet'])
df = df['snippet'].apply(pd.Series)
df = pd.concat([df, df_views], axis=1)
df = df.drop(['thumbnails', 'channelId', 'publishedAt'], axis=1)
df_filtered = df[df['channelTitle'] != 'KSW']
clean_df = standardize_text(df_filtered, "title")
clean_df["tokens"] = clean_df["title"].apply(tokenizer.tokenize)
clean_df.head(20)
clean_df['viewCount'].describe()
clean_df['publishTime'] = pd.to_datetime(clean_df['publishTime'])
clean_df['hour'] = clean_df['publishTime'].dt.hour
clean_df['dayPart'] = pd.cut(clean_df['hour'], bins=[0, 12, 18, 24], labels=['morning', 'afternoon', 'evening'], right=False)
clean_df['dayPart'] = preprocessing.LabelEncoder().fit_transform(clean_df['dayPart'])
clean_df['viewCount'] = clean_df['viewCount'].astype(float)
clean_df['view_level'] = pd.cut(clean_df['viewCount'], bins=[0, 10000, 25000, 50000, 75000, 100000, 150000, 200000, 10000000], labels=['0-10tys', '10-25tys', '25-50tys', '50-75tys', '75-100tys', '100-150tys', '150-200tys', '>200tys'], right=False)

sns.displot(clean_df['viewCount'])
plt.xlim(0,200000)
clean_df['view_level'][0:10]
encoder = preprocessing.LabelEncoder()
encoder.fit(clean_df['view_level'])
clean_df['view_level'] = encoder.transform(clean_df['view_level'])


clean_df['view_level'][0:10]
clean_df.head()
df_learn = clean_df.drop(['description', 'channelTitle', 'liveBroadcastContent', 'publishTime', 'hour'], axis=1)

vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w+')
X = vectorizer.fit_transform(df_learn['title'])
y = df_learn['view_level']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify = y)
print(X_train[:10])
def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.winter):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=20)
    plt.yticks(tick_marks, classes, fontsize=20)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", 
                 color="white" if cm[i, j] < thresh else "black", fontsize=40)
    
    plt.tight_layout()
    plt.ylabel('True label', fontsize=30)
    plt.xlabel('Predicted label', fontsize=30)

    return plt
ovo_clf = OneVsOneClassifier(LogisticRegression(C=30.0, solver='newton-cg', multi_class='multinomial',penalty='l2'))
ovo_clf.fit(X_train, y_train)

ovr_clf = OneVsRestClassifier(LogisticRegression(C=30.0, solver='newton-cg', multi_class='multinomial',penalty='l2'))
ovr_clf.fit(X_train, y_train)

y_predict_ovo = ovo_clf.predict(X_test)
y_predict_ovr = ovr_clf.predict(X_test)



# scoring model 
accuracy, precision, recall, f1 = get_metrics(y_test, y_predict_ovr)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))
# i'll be focusing on increasing precision cause I want to prove to cuustomer that he can feel confident about predictions (not to feel it is to optimistic)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_predict_ovr)

#np.fill_diagonal(norm_cm, 0)

cm_display = ConfusionMatrixDisplay(confusion_matrix = cm)

fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax = ax)
plt.show()


decoded_labels = encoder.inverse_transform(y_predict_ovr)

print("Odkodowane wartości:", decoded_labels)